N.b.: This notebook and other deployment workflows should end up in the cookiecutter to make one-click, documented workflows to automate important tasks for all future projects.

# Publish to PIP

Exectuable guide of how to publish your project on PyPi.

## Imports

In [1]:
import os
from os.path import dirname

import getpass
import configparser
import semantic_version

root_dir = dirname(dirname(os.getcwd()))
fn_setup_cfg = os.path.join(root_dir, 'setup.cfg')

## Functions

In [2]:
def get_config(fn_setup_cfg):
    config = configparser.ConfigParser()
    config.read(fn_setup_cfg)
    return config

def update_version(fn_setup_cfg, major=False, minor=False, patch=True, verbose=True):
    assert sum([major, minor, patch]) == 1, "Only one version number can be changed."
    
    # Load config
    config = get_config(fn_setup_cfg)
    
    # Get version
    v = semantic_version.Version(config['metadata'].get('version', '0.0.0'))
    
    # Update version
    if patch:
        config['metadata']['version'] = str(v.next_patch())
    elif minor:
        config['metadata']['version'] = str(v.next_minor())
    elif major:
        config['metadata']['version'] = str(v.next_major())
        
    if verbose:
        msg = """
        Version updated to: {}
        """.format(config['metadata']['version'])
        print(msg)
        
    # Save config
    with open(fn_setup_cfg, 'w') as f:
        config.write(f)
    return str(config['metadata']['version'])

PyPi
----

Make your project publicly available on the Python Package Index, [PyPi](https://pypi.org/). To achieve this, we need **remote dependency managment**, since you want your software to run without forcing the users to recreate your conda environments. All dependencies have to be managed, automatically, during installation. To make this work, we need to do some extra work.

We follow the steps as outlined in the most basic (and official) [PyPi tutorial](https://packaging.python.org/tutorials/packaging-projects/).

### Generate distribution archives

Generate distribution packages for the package. These are archives that are uploaded to the Package Index and can be installed by pip.

In [3]:
print("Root directory is {}".format(root_dir))

Root directory is /home/zissou/repos/elki_interface


We also update the version. PIP does not accept another identical file, since it keeps a complete history. Thus, we always must at least update the patch in order to push our thing to PIP. We do so automatically by adapting the versioning number in the `setup.cfg` file.

In [4]:
version = update_version(fn_setup_cfg, patch=True)


        Version updated to: 0.0.12
        


In [5]:
%%bash -s "$root_dir"

cd $1

python setup.py sdist bdist_wheel

running sdist
running egg_info
writing src/elki_interface.egg-info/PKG-INFO
writing dependency_links to src/elki_interface.egg-info/dependency_links.txt
writing requirements to src/elki_interface.egg-info/requires.txt
writing top-level names to src/elki_interface.egg-info/top_level.txt
reading manifest file 'src/elki_interface.egg-info/SOURCES.txt'
writing manifest file 'src/elki_interface.egg-info/SOURCES.txt'
running check
creating elki_interface-0.0.12
creating elki_interface-0.0.12/src
creating elki_interface-0.0.12/src/elki_interface
creating elki_interface-0.0.12/src/elki_interface.egg-info
copying files to elki_interface-0.0.12...
copying README.md -> elki_interface-0.0.12
copying setup.cfg -> elki_interface-0.0.12
copying setup.py -> elki_interface-0.0.12
copying src/elki_interface/Elki.py -> elki_interface-0.0.12/src/elki_interface
copying src/elki_interface/Hics.py -> elki_interface-0.0.12/src/elki_interface
copying src/elki_interface/__init__.py -> elki_interface-0.0.12/src/

### Upload to test-PyPi

After this, your package can be uploaded to the python package index. To see if it works on PyPi test server, the following

In [6]:
username = getpass.getpass()
pwd = getpass.getpass()

 ······
 ··········


In [7]:
%%bash --verbose -s "$root_dir" "$username" "$pwd" "$version"

cd $1

python -m twine upload --repository-url https://test.pypi.org/legacy/ dist/*$4* -u $2 -p $3

Uploading distributions to https://test.pypi.org/legacy/
Uploading elki_interface-0.0.12-py2.py3-none-any.whl
100%|██████████| 10.5k/10.5k [00:01<00:00, 7.51kB/s]
Uploading elki_interface-0.0.12.tar.gz
100%|██████████| 8.46k/8.46k [00:01<00:00, 6.97kB/s]

View at:
https://test.pypi.org/project/elki-interface/0.0.12/



cd $1

python -m twine upload --repository-url https://test.pypi.org/legacy/ dist/*$4* -u $2 -p $3


# Upload to Real PyPi

Test PyPi really does not work very well..

In [8]:
%%bash --verbose -s "$root_dir" "$username" "$pwd" "$version"

cd $1

python -m twine upload dist/*$4* -u $2 -p $3

Uploading distributions to https://upload.pypi.org/legacy/
Uploading elki_interface-0.0.12-py2.py3-none-any.whl
100%|██████████| 10.5k/10.5k [00:01<00:00, 7.00kB/s]
Uploading elki_interface-0.0.12.tar.gz
100%|██████████| 8.46k/8.46k [00:01<00:00, 7.78kB/s]

View at:
https://pypi.org/project/elki-interface/0.0.12/



cd $1

python -m twine upload dist/*$4* -u $2 -p $3
